In [1]:
import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial
import PIL
import PIL.Image
import random as python_random
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

np.random.seed(42) # note that you must use the same seed to ensure consistentcy in your training/validation/testing
tf.random.set_seed(42)



#### Questions:
# How does tf (under this method) store labels? Can I access them?
#    Look into how the resizing line is done
# What happens if I don't pass the y value into imageDataGenerator().flow()?
# is this enough work? (oversampling, data augmentation, adusting the prediction wieghts)
# multiple expert -  3 different model (majority voting at the end to make predictions) - cost trade off for computing
# - talk about costs (training time, different work, etc)
# - accuracy vs tradeoffs (latency of making one prediction)

## Load Data

In [2]:
# if the data above isn't run, run this

data_path = "../dl_data/"
class_names = os.listdir(data_path)
class_dist = {}
for c in class_names:
    class_dist[c] = len(os.listdir(data_path + c))

In [3]:
class_names

['Covid_img', 'Viral_img', 'Normal_img']

In [4]:
from sklearn.datasets import load_files 
from keras.utils import np_utils

from keras.preprocessing import image

#### calculate class weights

# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
total = sum(class_dist.values())
weight_for_0 = (1 / class_dist[class_names[0]]) * (total / 2.0)
weight_for_1 = (1 / class_dist[class_names[1]]) * (total / 2.0)
weight_for_2 = (1 / class_dist[class_names[2]]) * (total / 2.0)

class_weights = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

# directories
data_dir = "../dl_data"
# test_dir = "../test_data"
HOLD_dir = "../HOLD_data"


#### save out augmented data for visualization

# ## first delete any existing files
# aug_dir = '../augmented_data'
# aug_files = os.listdir(aug_dir)
# for f in aug_files:
#     os.remove(aug_dir + '/' + f)

    
batch_size = 32;
# IMPORTANT: Depends on what pre-trained model you choose, you will need to change these dimensions accordingly
img_height = 224; 
img_width = 224;
    

# data augmentation (for training only)
train_data_gen = ImageDataGenerator(rescale=1./255,
                                    zoom_range= 0.3, 
                                    horizontal_flip= True, 
                                    shear_range= 0.2,
                                    rotation_range = 30,
                                    validation_split=0.2

                                    )



train_ds = train_data_gen.flow_from_directory(
    directory = data_dir,
    target_size=(img_height, img_width),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    seed=42,
#     save_to_dir=aug_dir,
#     save_prefix='aug',
#     save_format='png',
    follow_links=False,
    subset='training',
    interpolation='nearest'
)

validation_ds = train_data_gen.flow_from_directory(
    directory=data_dir,  # same directory because we are splitting the data here
    follow_links=False,
    subset='validation',
    interpolation='nearest',
    target_size=(img_height, img_width), 
    class_mode='categorical',
    shuffle=False,
    seed=42,
    batch_size=batch_size
)

class_ind = (train_ds.class_indices)

test_data_gen = ImageDataGenerator(rescale=1./255)


# holdout data
HOLD_ds = test_data_gen.flow_from_directory(directory=HOLD_dir, 
                                         target_size=(img_height, img_width), 
                                         class_mode='categorical',
                                         shuffle=False,
                                         seed=42,
                                         batch_size=batch_size)

Found 10912 images belonging to 3 classes.
Found 2726 images belonging to 3 classes.
Found 1514 images belonging to 3 classes.


In [5]:
class_ind

# scikitlearn funciton for recall/precision etc. scikitlearn.metrics
#train on accuracy

{'Covid_img': 0, 'Normal_img': 1, 'Viral_img': 2}

In [6]:
## this is just a bug fix, hopefully I won't need to use it again.

# fi = os.listdir(aug_dir + '/' + os.listdir(aug_dir)[0])
# for f in fi:
#     os.remove(aug_dir + '/' + os.listdir(aug_dir)[0] + '/' + f)

# os.rmdir(aug_dir + '/' + os.listdir(aug_dir)[0])

In [7]:
# set checkpoint to resume training if it stops unexpectedly
checkpoint_path = "../checkpoints/training_ROUND3/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [8]:
ds_size_1 = (224, 224)
# train_ds_1 = train_ds.map(lambda image, label: (tf.image.resize(image, ds_size_1), label))
# validation_ds_1 = validation_ds.map(lambda image, label: (tf.image.resize(image, ds_size_1), label))

train_ds_1 = train_ds
validation_ds_1 = validation_ds


base_model_2 = keras.applications.ResNet50(weights='imagenet', include_top=False)
n_classes = 3

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# Rebuild top
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(base_model_2.output)
x = tf.keras.layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
# x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(3, activation="softmax", name="pred")(x) # match number of classes

model_2 = keras.models.Model(inputs=base_model_2.input,
                           outputs=outputs)

2022-04-26 12:51:12.512087: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-26 12:51:14.388443: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38397 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:bd:00.0, compute capability: 8.0


In [9]:
# train up the top layer first

for layer in base_model_2.layers:
    layer.trainable = False

recall = tf.keras.metrics.Recall()
optimizer = keras.optimizers.Adam(learning_rate=0.01, decay=0.01)
model_2.compile(loss="categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model_2.fit(train_ds_1,
                    validation_data=validation_ds_1,
#                     class_weight=class_weights,
                    epochs=3, callbacks=[callback,cp_callback])

Epoch 1/3


2022-04-26 11:01:02.353890: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


  1/341 [..............................] - ETA: 53:42 - loss: 1.1885 - accuracy: 0.3125

2022-04-26 11:01:07.336341: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


341/341 [==============================] - ETA: 0s - loss: 0.9200 - accuracy: 0.6267
Epoch 1: saving model to ../checkpoints/training_ROUND3/cp.ckpt
341/341 [==============================] - 148s 406ms/step - loss: 0.9200 - accuracy: 0.6267 - val_loss: 0.8385 - val_accuracy: 0.6728
Epoch 2/3
341/341 [==============================] - ETA: 0s - loss: 0.8547 - accuracy: 0.6682
Epoch 2: saving model to ../checkpoints/training_ROUND3/cp.ckpt
341/341 [==============================] - 135s 397ms/step - loss: 0.8547 - accuracy: 0.6682 - val_loss: 0.8209 - val_accuracy: 0.6724
Epoch 3/3
341/341 [==============================] - ETA: 0s - loss: 0.8428 - accuracy: 0.6719
Epoch 3: saving model to ../checkpoints/training_ROUND3/cp.ckpt
341/341 [==============================] - 136s 400ms/step - loss: 0.8428 - accuracy: 0.6719 - val_loss: 0.8336 - val_accuracy: 0.6728


In [10]:
# opt = tf.keras.optimizers.Adam(0.1)
# net = Net()
# dataset = toy_dataset()
# iterator = iter(dataset)
# ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=net, iterator=iterator)
# manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=3)

# train_and_checkpoint(net, manager)

In [ ]:
# train all the layers together for a bit with a much lower learning rate

for layer in base_model_2.layers[-20:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

recall = tf.keras.metrics.Recall()
optimizer = keras.optimizers.Adam(learning_rate=0.0004, decay=0.001)
model_2.compile(loss="categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model_2.fit(train_ds_1,
                    validation_data=validation_ds_1,
#                     class_weight=class_weights,
                    epochs=50, callbacks=[callback,cp_callback])

Epoch 1/50


2022-04-26 12:51:22.772587: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2022-04-26 12:51:27.915303: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


341/341 [==============================] - ETA: 0s - loss: 1.0909 - accuracy: 0.5589
Epoch 1: saving model to ../checkpoints/training_ROUND3/cp.ckpt
341/341 [==============================] - 158s 429ms/step - loss: 1.0909 - accuracy: 0.5589 - val_loss: 8.1972 - val_accuracy: 0.6728
Epoch 2/50
341/341 [==============================] - ETA: 0s - loss: 0.9076 - accuracy: 0.6552
Epoch 2: saving model to ../checkpoints/training_ROUND3/cp.ckpt
341/341 [==============================] - 138s 405ms/step - loss: 0.9076 - accuracy: 0.6552 - val_loss: 283.9296 - val_accuracy: 0.2384
Epoch 3/50
341/341 [==============================] - ETA: 0s - loss: 0.9044 - accuracy: 0.6603
Epoch 3: saving model to ../checkpoints/training_ROUND3/cp.ckpt
341/341 [==============================] - 137s 401ms/step - loss: 0.9044 - accuracy: 0.6603 - val_loss: 0.8263 - val_accuracy: 0.6728
Epoch 4/50
341/341 [==============================] - ETA: 0s - loss: 0.8883 - accuracy: 0.6643
Epoch 4: saving model to ../

In [ ]:
# # train all the layers together for a bit with a much lower learning rate

# for layer in base_model_2.layers[-20:]:
#     if not isinstance(layer, tf.keras.layers.BatchNormalization):
#         layer.trainable = True

# recall = tf.keras.metrics.Recall()
# optimizer = keras.optimizers.Adam(learning_rate=0.0004, decay=0.001)
# model_2.compile(loss="categorical_crossentropy", optimizer=optimizer,
#               metrics=["accuracy"])
# history = model_2.fit(train_ds_1,
#                     validation_data=validation_ds_1,
# #                     class_weight=class_weights,
#                     epochs=1, callbacks=[callback,cp_callback])

In [ ]:
# save the model
model_2.save('saved_models/model_ROUND3') # change this path to save a new version

In [ ]:
# # if you need to use the checkpoint, use this code
# # source: https://www.tensorflow.org/tutorials/keras/save_and_load#checkpoint_callback_options

# latest = tf.train.latest_checkpoint(checkpoint_dir)
# latest

# # Create a new model instance
# model_2 = create_model()

# # Load the previously saved weights
# model_2.load_weights(latest)

# # Re-evaluate the model
# loss, acc = model_2.evaluate(validation_ds_1 verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
! ls 

To Do:

Oversampling/Data Augmentation:

1. start a new file with clear labels, resampling, augmented data
2. Train the model the same way
3. Save model and create confusion matrix in this file (or seperate file)

Prediction weights
1. When predicting classes, change wieghts until we get 100% for covid cases
2. Change to proportional CM instead of just numeric?

Recall and F-score as metric?